# Clustering Crypto

In [61]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [84]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
iris_df = pd.read_csv(file_path)
iris_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [85]:
# Keep all the cryptocurrencies that are being traded.
false_drop = iris_df['IsTrading'] != False
iris_df = iris_df.loc[false_drop]
iris_df.count()

Unnamed: 0         1144
CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [86]:
# Keep all the cryptocurrencies that have a working algorithm.
iris_df= iris_df.set_index('Unnamed: 0')

In [96]:
# Remove the "IsTrading" column. 
iris_df = iris_df.drop(columns=["IsTrading"])

In [97]:
# Remove rows that have at least 1 null value.
iris_df = iris_df.dropna()
iris_df.count()

CoinName           685
Algorithm          685
ProofType          685
TotalCoinsMined    685
TotalCoinSupply    685
dtype: int64

In [98]:
# Keep the rows where coins are mined.
iris_df = iris_df.dropna()
iris_df.count()

CoinName           685
Algorithm          685
ProofType          685
TotalCoinsMined    685
TotalCoinSupply    685
dtype: int64

In [99]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = iris_df.drop(['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'], axis=1)
names_df.head()

,CoinName
Unnamed: 0,
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin


In [100]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = iris_df.drop(['CoinName'], axis=1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000


In [101]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
# Standardize the data with StandardScaler().
iris_scaled = StandardScaler().fit_transform(X)
print(iris_scaled[0:5])

[[-0.10282804 -0.03823841 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.05411338 -0.07664017 -0.03823596 -0.05411338 -0.05411338 -0.03823596
  -0.03823596 -0.18216065 -0.05411338 -0.03823596 -0.03823596 -0.08574929
  -0.03823596 -0.10160947 -0.06632365 -0.03823596 -0.03823596 -0.1642757
  -0.03823596 -0.03823596 -0.13908716 -0.03823596 -0.03823596 -0.07664017
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.06632365 -0.03823596
  -0.07664017 -0.08574929 -0.07664017 -0.03823596 -0.03823596 -0.12775161
  -0.1335313  -0.13908716 -0.03823596 -0.05411338 -0.03823596 -0.06632365
  -0.1689039  -0.03823596 -0.03823596 -0.03823596 -0.07664017 -0.17342199
  -0.33468341 -0.03823596 -0.08574929 -0.06632365 -0.05411338 -0.03823596
   1.42042992 -0.06632365 -0.03823596 -0.03823596 -0.06632365 -0.06632365
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.03823596 -0.41586681 -0.03823596 -0.19054822 -0.03823596 -0.10870529
  -0.07664017 -0.09400279 -0.03823596 -

### Deliverable 2: Reducing Data Dimensions Using PCA

In [93]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)


In [119]:
# Create a DataFrame with the three principal components.
iris_pca = pca.fit_transform(iris_scaled)
pcs_df = pd.DataFrame(data=iris_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head()
pcs_df.count()

PC 1    685
PC 2    685
PC 3    685
dtype: int64

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [106]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


C:\Users\JohnPaul.Kivlin\Anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [105]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=5)
model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 1
 0 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 0
 1 0 1 0 1 1 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0
 1 1 1 0 0 1 0 1 1 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0
 0 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1
 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0
 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1
 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0 1
 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 1
 1 0 1 0 0 1 0 0 0 0 0 0 

In [154]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
#frames = [crypto_df, pcs_df]
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
CoinName = names_df["CoinName"]
clustered_df = clustered_df.join(CoinName)


#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head()
#clustered_df['TotalCoinSupply'].value_counts()

(685, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.307591,1.261662,-0.463406,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.293082,1.260056,-0.464081,404Coin,1
808,SHA-256,PoW/PoS,0.000000e+00,0,-0.169777,0.866374,-0.309102,808,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.238985,2.013634,-0.521620,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.209523,-1.439863,0.125290,Bitcoin,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [143]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", hover_data=["Algorithm"], hover_name="CoinName", color="class", symbol="class",width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [155]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [163]:
# Print the total number of tradable cryptocurrencies.
count = clustered_df['CoinName'].count()
print(f"There are {count} tradable crytocurrencies" )

There are 685 tradable crytocurrencies


In [164]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler()
scaler.fit(clustered_df)

ValueError: could not convert string to float: 'Scrypt'

In [21]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [22]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)